In [12]:
import pandas as pd
from sklearn.linear_model import RidgeClassifier
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score

In [13]:
df = pd.read_csv("nba_games.csv", index_col=0)
df = df.sort_values("date").reset_index(drop=True)

In [14]:
del df["mp.1"]
del df["mp_opp.1"]
del df["index_opp"]

In [15]:
def add_target(group):
    group["target"] = group["won"].shift(-1)
    return group

In [16]:
df = df.groupby("team", group_keys=False).apply(add_target).reset_index(drop=True)
df["target"][pd.isnull(df["target"])] = 2
df["target"] = df["target"].astype(int, errors="ignore")

/var/folders/cf/2q1x6qbd7t10wbl_xmlrblfc0000gn/T/ipykernel_70592/3178735140.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  group["target"] = group["won"].shift(-1)
/var/folders/cf/2q1x6qbd7t10wbl_xmlrblfc0000gn/T/ipykernel_70592/3178735140.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  group["target"] = group["won"].shift(-1)
/var/folders/cf/2q1x6qbd7t10wbl_xmlrblfc0000gn/T/ipykernel_70592/3178735140.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many tim

In [17]:
nulls = pd.isnull(df).sum()
nulls = nulls[nulls > 0]
valid_columns = df.columns[~df.columns.isin(nulls.index)]
df = df[valid_columns].copy()

In [18]:
removed_columns = ["season", "date", "won", "target", "team", "team_opp"]
selected_columns = df.columns[~df.columns.isin(removed_columns)]

scaler = MinMaxScaler()
df[selected_columns] = scaler.fit_transform(df[selected_columns])

In [19]:
rr = RidgeClassifier(alpha=1)
split = TimeSeriesSplit(n_splits=3)

In [20]:
def backtest(data, model, predictors, start=2, step=1):
    all_predictions = []
    
    seasons = sorted(data["season"].unique())
    
    for i in range(start, len(seasons), step):
        season = seasons[i]
        train = data[data["season"] < season]
        test = data[data["season"] == season]
        
        model.fit(train[predictors], train["target"])
        
        preds = model.predict(test[predictors])
        preds = pd.Series(preds, index=test.index)
        combined = pd.concat([test["target"], preds], axis=1)
        combined.columns = ["actual", "prediction"]
        
        all_predictions.append(combined)
    return pd.concat(all_predictions)

In [21]:
predictors = list(selected_columns)
predictions = backtest(df, rr, predictors)

In [22]:
wynik = accuracy_score(predictions["actual"], predictions["prediction"])
print(f"wynik to {wynik*100:.4f}%" + " dokładności osiągnięte przez model bez selekcji cech")

wynik to 53.330138955438436% dokładności osiągnięte przez model bez selekcji cech


In [23]:
df.groupby(["home"]).apply(lambda x: x[x["won"] == 1].shape[0] / x.shape[0])

/var/folders/cf/2q1x6qbd7t10wbl_xmlrblfc0000gn/T/ipykernel_70592/330557953.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df.groupby(["home"]).apply(lambda x: x[x["won"] == 1].shape[0] / x.shape[0])


home
0.0    0.428314
1.0    0.571686
dtype: float64

In [24]:
print(f"wynik to {0.571686*100:.4f}%" + " dokładności oceniając, czy drużyna grała na swoim stadionie")

wynik to 57.1686% dokładności oceniając, czy drużyna grała na swoim stadionie
